In [46]:
#pip install psycopg2
#pip install pyodbc
#pip install csv
#pip install regex
import psycopg2
import pyodbc
import csv
import re

# Kết nối đến PostgreSQL
postgres_conn = psycopg2.connect(
    host="103.130.215.192",
    port="5432",
    database="northwind_pg",
    user="postgres",
    password="admin12345"
)

# Kết nối đến SQL Server
sql_server_conn = pyodbc.connect(
    "Driver={SQL Server};"
    #đây là ip 192.168.100.27 local sql server của máy em
    "Server=INTELPC;"
    "Database=northwind_sql3;"
    "UID=root;"
    "PWD=admin123;"
)

# Lấy danh sách các bảng trong PostgreSQL
postgres_cursor = postgres_conn.cursor()
postgres_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
postgres_tables = postgres_cursor.fetchall()

# Lấy danh sách các bảng trong SQL Server
sql_server_cursor = sql_server_conn.cursor()
sql_server_cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE';")
sql_server_tables = sql_server_cursor.fetchall()

In [47]:
# so sánh cấu trúc bảng của 2 hệ cơ sở dữ liệu
structs_list = []

# So sánh dữ liệu trong các bảng
for postgres_table in postgres_tables:
    postgres_table_name = postgres_table[0]
    sql_server_table_name = None

    # Tìm bảng tương ứng trong SQL Server
    for sql_server_table in sql_server_tables:
        if sql_server_table[0] == 'Order Details':
            sql_snack_tab = 'order_details'
        else:
            sql_snack_tab = re.sub(r'(?<!^)(?=[A-Z])', '_', sql_server_table[0]).lower()
        if sql_snack_tab.lower() == postgres_table_name.lower():
            sql_server_table_name = sql_snack_tab
            core_sql_tb = sql_server_table[0] 
            break
    
    if sql_server_table_name:
        # Lấy danh sách các cột trong PostgreSQL
        postgres_cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{postgres_table_name}' ORDER BY ordinal_position DESC;")
        postgres_columns = postgres_cursor.fetchall()

        # Lấy danh sách các cột trong SQL Server
        sql_server_cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{core_sql_tb}' ORDER BY ordinal_position DESC;")
        sql_server_columns = sql_server_cursor.fetchall()

        # print(postgres_columns)
        # print(sql_server_columns)

        for postgres_column, sql_server_column in zip(postgres_columns, sql_server_columns):
            postgres_column_name = postgres_column[0]
            sql_server_column_name = sql_server_column[0]

            postgres_data_type = None
            sql_data_type = None
            #kiểm tra xem kiểu của cột dữ liệu có trống ko thì lấy kiểu dữ liệu của cột
            if len(postgres_column) > 1:
                postgres_data_type = postgres_column[1]
            if len(sql_server_column) > 1:
                sql_data_type = sql_server_column[1]
            structs_list.append({
                    "table_name": postgres_table_name,
                    "column_name": postgres_column_name,
                    "postgres_type": postgres_data_type,
                    "sql_server_type": sql_data_type
                })
    # Xuất kiểu dữ liệu ra file CSV
    if structs_list:
        with open("compare_struct.csv", "w", newline="") as csvfile:
            fieldnames = ["table_name", "column_name", "postgres_type", "sql_server_type"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for data in structs_list:
                writer.writerow(data)
        print("Đã xuất dữ liệu ra file compare_struct.csv")
    else:
        print("Không có dữ liệu để xuất ra file CSV.")

    # # Đóng kết nối
    # postgres_cursor.close()
    # postgres_conn.close()
    # sql_server_cursor.close()
    # sql_server_conn.close()


Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv
Đã xuất dữ liệu ra file compare_struct.csv


In [48]:
#So sánh số lượng record
with open('record_counts.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Table Name', 'PostgreSQL Count', 'MS SQL Server Count'])

    # Duyệt qua từng bảng và đếm số lượng bản ghi
    for postgres_table in postgres_tables:
        postgres_table_name = postgres_table[0]
        
        tbl_name_trans = None
        for sql_server_table in sql_server_tables:
            if sql_server_table[0] == 'Order Details':
                sql_snack_tab = 'order_details'
            else:
                sql_snack_tab = re.sub(r'(?<!^)(?=[A-Z])', '_', sql_server_table[0]).lower()
    
            if sql_snack_tab.lower() == postgres_table_name.lower():
                tbl_name_trans = sql_snack_tab
                core_sql_tb = sql_server_table[0] 
                break

        if tbl_name_trans:
            postgres_cursor.execute(f"SELECT COUNT(*) FROM {tbl_name_trans}")
            record_count_postgres = postgres_cursor.fetchone()[0]
            
            sql_server_cursor.execute(f"SELECT COUNT(*) FROM [{core_sql_tb}]")
            record_count_mssql = sql_server_cursor.fetchone()[0]

            writer.writerow([postgres_table_name, record_count_postgres, record_count_mssql])
        else:
            print(postgres_table_name)
            postgres_cursor.execute(f"SELECT COUNT(*) FROM {postgres_table_name}")
            record_count_postgres = postgres_cursor.fetchone()[0]
            writer.writerow([postgres_table_name, record_count_postgres, 0])

# Đóng kết nối
# postgres_cursor.close()
# postgres_conn.close()
# sql_server_cursor.close()
# sql_server_conn.close()

us_states
